<a href="https://colab.research.google.com/github/jeniferGoncalvesDaSilvaDev/NPL_previsao_demand-a_elasticidade/blob/main/Npl_previsao_demanda_elasticidade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline
from math import isclose

# Pipeline NLP
classifier = pipeline("text-classification", model="distilbert-base-uncased")

label_mapping = {
    "LABEL_0": "Alta",
    "LABEL_1": "Média",
    "LABEL_2": "Baixa"
}

# Predição baseada em texto (NLP)
def predict_demand(text: str) -> str:
    result = classifier(text)[0]
    predicted_label = result["label"]
    return label_mapping.get(predicted_label, "Desconhecida")

# Cálculo da elasticidade
def calcular_elasticidade_preco_demanda(preco_inicial: float, preco_final: float,
                                        demanda_inicial: float, demanda_final: float) -> float:
    try:
        delta_q = demanda_final - demanda_inicial
        delta_p = preco_final - preco_inicial

        if isclose(preco_inicial, 0.0) or isclose(demanda_inicial, 0.0):
            raise ValueError("Preço ou demanda inicial não pode ser zero.")

        percentual_q = delta_q / demanda_inicial
        percentual_p = delta_p / preco_inicial

        epd = percentual_q / percentual_p
        return round(epd, 4)
    except ZeroDivisionError:
        return float('inf')
    except Exception as e:
        return str(e)

# Classificação com base no valor de EPD
def classificar_elasticidade_por_epd(epd: float) -> str:
    if isinstance(epd, str):
        return "Erro"
    if epd < -1:
        return "Alta"
    elif -1 <= epd <= -0.5:
        return "Média"
    else:
        return "Baixa"

# Função integrada: calcula e classifica
def prever_elasticidade_por_numeros(preco_inicial: float, preco_final: float,
                                    demanda_inicial: float, demanda_final: float) -> dict:
    epd = calcular_elasticidade_preco_demanda(preco_inicial, preco_final, demanda_inicial, demanda_final)
    classificacao = classificar_elasticidade_por_epd(epd)
    return {
        "epd_value": epd,
        "classificacao": classificacao
    }

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
import torch

# Apply dynamic quantization to the model
quantized_model = torch.quantization.quantize_dynamic(
    classifier.model, {torch.nn.Linear}, dtype=torch.qint8
)

# Salvar o estado do modelo quantizado em um arquivo .pt
torch.save(quantized_model.state_dict(), 'quantized_classifier_model.pt')

print("Estado do modelo quantizado salvo com sucesso em quantized_classifier_model.pt")

Estado do modelo quantizado salvo com sucesso em quantized_classifier_model.pt


In [9]:
from transformers import pipeline
from math import isclose

# Pipeline NLP
classifier = pipeline("text-classification", model="distilbert-base-uncased")

label_mapping = {
    "LABEL_0": "Alta",
    "LABEL_1": "Média",
    "LABEL_2": "Baixa"
}

# Predição baseada em texto (NLP)
def predict_demand(text: str) -> str:
    result = classifier(text)[0]
    predicted_label = result["label"]
    return label_mapping.get(predicted_label, "Desconhecida")

# Cálculo da elasticidade
def calcular_elasticidade_preco_demanda(preco_inicial: float, preco_final: float,
                                        demanda_inicial: float, demanda_final: float) -> float:
    try:
        delta_q = demanda_final - demanda_inicial
        delta_p = preco_final - preco_inicial

        if isclose(preco_inicial, 0.0) or isclose(demanda_inicial, 0.0):
            raise ValueError("Preço ou demanda inicial não pode ser zero.")

        percentual_q = delta_q / demanda_inicial
        percentual_p = delta_p / preco_inicial

        epd = percentual_q / percentual_p
        return round(epd, 4)
    except ZeroDivisionError:
        return float('inf')
    except Exception as e:
        return str(e)

# Classificação com base no valor de EPD
def classificar_elasticidade_por_epd(epd: float) -> str:
    if isinstance(epd, str):
        return "Erro"
    if epd < -1:
        return "Alta"
    elif -1 <= epd <= -0.5:
        return "Média"
    else:
        return "Baixa"

# Função integrada: calcula e classifica via números
def prever_elasticidade_por_numeros(preco_inicial: float, preco_final: float,
                                    demanda_inicial: float, demanda_final: float) -> dict:
    epd = calcular_elasticidade_preco_demanda(preco_inicial, preco_final, demanda_inicial, demanda_final)
    classificacao = classificar_elasticidade_por_epd(epd)
    return {
        "epd_value": epd,
        "classificacao": classificacao
    }

# Função híbrida: análise por texto + números
def analisar_demanda_completa(texto: str, preco_inicial: float, preco_final: float,
                               demanda_inicial: float, demanda_final: float) -> dict:
    # Resultado baseado em NLP (texto)
    resultado_texto = predict_demand(texto)

    # Resultado baseado em números (EPD)
    resultado_numerico = prever_elasticidade_por_numeros(preco_inicial, preco_final, demanda_inicial, demanda_final)

    # Comparação e consistência
    consistencia = resultado_texto == resultado_numerico["classificacao"]

    return {
        "analise_texto": resultado_texto,
        "analise_numerica": resultado_numerico,
        "consistente": consistencia
    }

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


In [10]:
# Example usage of the integrated function
text_example = "The demand for this product is high."
preco_inicial_example = 10.0
preco_final_example = 12.0
demanda_inicial_example = 100.0
demanda_final_example = 80.0

analise_completa = analisar_demanda_completa(
    text_example,
    preco_inicial_example,
    preco_final_example,
    demanda_inicial_example,
    demanda_final_example
)

print("Análise completa da demanda:")
print(analise_completa)

Análise completa da demanda:
{'analise_texto': 'Alta', 'analise_numerica': {'epd_value': -1.0, 'classificacao': 'Média'}, 'consistente': False}


In [ ]:
# Exemplos de uso da função de predição baseada em texto
text_example_1 = "Este produto tem alta demanda no mercado."
text_example_2 = "A procura por este serviço é moderada."
text_example_3 = "Ninguém está comprando este item."

print(f"Previsão para '{text_example_1}': {predict_demand(text_example_1)}")
print(f"Previsão para '{text_example_2}': {predict_demand(text_example_2)}")
print(f"Previsão para '{text_example_3}': {predict_demand(text_example_3)}")

Previsão para 'Este produto tem alta demanda no mercado.': Alta
Previsão para 'A procura por este serviço é moderada.': Alta
Previsão para 'Ninguém está comprando este item.': Alta


In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import Optional

# Assuming predict_demand and prever_elasticidade_por_numeros are defined in the notebook
# from models.dummy_model import predict_demand, prever_elasticidade_por_numeros

app = FastAPI(title="Elasticidade Demanda API Inteligente")

# Entrada unificada
class ElasticityRequest(BaseModel):
    text: Optional[str] = None
    preco_inicial: Optional[float] = None
    preco_final: Optional[float] = None
    demanda_inicial: Optional[float] = None
    demanda_final: Optional[float] = None

# Saída unificada
class ElasticityResponse(BaseModel):
    method: str
    predicted_elasticity: str
    epd_value: Optional[float] = None

@app.post("/predict-elasticity", response_model=ElasticityResponse)
async def predict_elasticity(data: ElasticityRequest):
    # Caso 1: usar NLP se só tiver texto
    if data.text and not any([data.preco_inicial, data.preco_final, data.demanda_inicial, data.demanda_final]):
        prediction = predict_demand(data.text)
        return ElasticityResponse(method="NLP", predicted_elasticity=prediction)

    # Caso 2: usar fórmula se dados numéricos forem enviados
    elif all(v is not None for v in [data.preco_inicial, data.preco_final, data.demanda_inicial, data.demanda_final]):
        resultado = prever_elasticidade_por_numeros(
            data.preco_inicial, data.preco_final, data.demanda_inicial, data.demanda_final
        )
        return ElasticityResponse(
            method="Fórmula",
            predicted_elasticity=resultado["classificacao"],
            epd_value=resultado["epd_value"]
        )

    # Caso inválido
    raise HTTPException(
        status_code=400,
        detail="Você deve enviar ou apenas 'text', ou todos os campos: preco_inicial, preco_final, demanda_inicial e demanda_final."
    )